In [1]:
import tensorflow as tf
import wandb
import dlomix
import os
import dlomix.losses
from dlomix.data import load_processed_dataset
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
from dlomix.losses import masked_spectral_distance

2024-07-25 10:19:37.187693: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-25 10:19:37.187732: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-25 10:19:37.188863: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-25 10:19:37.196449: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-25 10:19:39.109627: W tensorflow/compiler/tf2


Avaliable feature extractors are (use the key of the following dict and pass it to features_to_extract in the Dataset Class):
{
   "atom_count": "Atom count of PTM.",
   "delta_mass": "Delta mass of PTM.",
   "mod_gain": "Gain of atoms due to PTM.",
   "mod_loss": "Loss of atoms due to PTM.",
   "red_smiles": "Reduced SMILES representation of PTM."
}.
When writing your own feature extractor, you can either
    (1) use the FeatureExtractor class or
    (2) write a function that can be mapped to the Hugging Face dataset.
In both cases, you can access the parsed sequence information from the dataset using the following keys, which all provide python lists:
    - _parsed_sequence: parsed sequence
    - _n_term_mods: N-terminal modifications
    - _c_term_mods: C-terminal modifications



In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = '3'

In [3]:
MODEL_DIR = '/cmnfs/proj/bmpc_dlomix/models/baseline_models/noptm_baseline_full_bs1024_unmod_extended/'
RUN_NAME = '7ef3360f-2349-46c0-a905-01187d4899e2'
model = tf.keras.models.load_model(MODEL_DIR + RUN_NAME + '.keras',)
print(model.summary())

2024-07-25 10:19:45.558569: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7505 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080, pci bus id: 0000:82:00.0, compute capability: 6.1


Model: "prosit_intensity_predictor"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  464       
                                                                 
 sequential (Sequential)     (None, 30, 512)           1996800   
                                                                 
 sequential_1 (Sequential)   multiple                  4608      
                                                                 
 sequential_2 (Sequential)   (None, 29, 512)           1576806   
                                                                 
 encoder_att (AttentionLaye  multiple                  542       
 r)                                                              
                                                                 
 sequential_3 (Sequential)   multiple                  0         
                                        

In [4]:
ds = load_processed_dataset('/cmnfs/proj/bmpc_dlomix/datasets/processed/noptm_baseline_small_duplicated_ions')
ds.batch_size = 1

In [5]:
for b, l in ds.tensor_train_data:
    batch = b
    break

In [10]:
# test with real models
og_model = tf.keras.models.load_model(MODEL_DIR + RUN_NAME + '.keras')

In [11]:
import sys
sys.path.append('../bmpc_shared_scripts/refinement_transfer_learning')

In [12]:
from change_layers import change_output_layer
import keras.backend as K
import keras

In [13]:
class DoubleRegressorWeights(keras.initializers.Initializer):

    def __init__(self, weights):
      self.weights = K.concatenate([weights, weights], axis=1)
      # self.weights = [weights, weights]

    def __call__(self, shape, dtype=None):
      return self.weights

    def get_config(self):  # To support serialization
      return {'weights': self.weights, 'shape': self.weights.shape}



class DoubleRegressorBias(keras.initializers.Initializer):

    def __init__(self, bias):
      self.bias = K.concatenate([bias, bias], axis=0)
      # self.bias = [bias, bias]

    def __call__(self, shape, dtype=None):
      return self.bias

    def get_config(self):  # To support serialization
      return {'bias': self.bias, 'shape': self.bias.shape}

In [14]:
changed_model = tf.keras.models.load_model(MODEL_DIR + RUN_NAME + '.keras')

In [15]:
old_weights = changed_model.regressor.get_layer('time_dense').get_weights()[0]
old_bias = changed_model.regressor.get_layer('time_dense').get_weights()[1]

In [16]:
changed_model.len_fion = 3 * 4
changed_model.regressor = tf.keras.Sequential(
        [
            tf.keras.layers.TimeDistributed(
                tf.keras.layers.Dense(changed_model.len_fion,
                                      kernel_initializer=DoubleRegressorWeights(old_weights),
                                      bias_initializer=DoubleRegressorBias(old_bias)), name='time_dense'
                ), 
            tf.keras.layers.LeakyReLU(name='activation'), 
            tf.keras.layers.Flatten(name='out')], 
        name='regressor'
        )

In [17]:
changed_model.compile(tf.keras.optimizers.Adam(learning_rate=0.0001), loss=masked_spectral_distance)

In [19]:
changed_model(batch).shape

TensorShape([1, 348])

In [22]:
og_dense_weights = og_model.regressor.get_layer('time_dense').get_weights()[0]
changed_dense_weights = changed_model.regressor.get_layer('time_dense').get_weights()[0]

In [24]:
og_dense_weights.shape

(512, 6)

In [26]:
np.allclose(og_dense_weights, changed_dense_weights[:, 6:])

True

In [28]:
og_output = og_model(batch)
changed_output = changed_model(batch)
og_output.shape, changed_output.shape

(TensorShape([1, 174]), TensorShape([1, 348]))

In [32]:
np.allclose(og_output[:, 6:12], changed_output[:, 12:18])

True